![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/work-with-data/dataprep/how-to-guides/split-column-by-example.png)

# Split column by example


DataPrep also offers you a way to easily split a column into multiple columns.
The SplitColumnByExampleBuilder class lets you generate a proper split program that will work even when the cases are not trivial, like in example below.

In [ ]:
import azureml.dataprep as dprep

In [ ]:
dflow = dprep.read_lines(path='../data/crime.txt')
df = dflow.head(10)

In [ ]:
df['Line'].iloc[0]

As you can see above, you can't split this particular file by space character as it will create too many columns.
That's where split_column_by_example could be quite useful.

In [ ]:
builder = dflow.builders.split_column_by_example('Line', keep_delimiters=True)

In [ ]:
builder.preview()

Couple things to take note of here. No examples were given, and yet DataPrep was able to generate quite reasonable split program. 
We have passed keep_delimiters=True so we can see all the data split into columns. In practice, though, delimiters are rarely useful, so let's exclude them.

In [ ]:
builder.keep_delimiters = False
builder.preview()

This looks pretty good already, except that one case number is split into 2 columns. Taking the first row as an example, we want to keep case number as "HY329907" instead of "HY" and "329907" seperately.  
If we request generation of suggested examples we will get a list of examples that require input.

In [ ]:
suggestions = builder.generate_suggested_examples()
suggestions

In [ ]:
suggestions.iloc[0]['Line']

Having retrieved source value we can now provide an example of desired split.
Notice that we chose not to split date and time but rather keep them together in one column.

In [ ]:
builder.add_example(example=(suggestions['Line'].iloc[0], ['10140490','HY329907','7/5/2015  23:50','050XX N NEWLAND AVE','820','THEFT']))

In [ ]:
builder.preview()

As we can see from the preview, some of the crime types (`Line_6`) do not show up as expected. Let's try to add one more example. 

In [ ]:
builder.add_example(example=(df['Line'].iloc[1],['10139776','HY329265','7/5/2015  23:30','011XX W MORSE AVE','460','BATTERY']))
builder.preview()

This looks just like what we need. Let's get a dataflow with splited columns and drop original column.

In [ ]:
dflow = builder.to_dataflow()
dflow = dflow.drop_columns(['Line'])
dflow.head(5)

Now we have successfully split the data into useful columns through examples.